In [10]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import pandas

device = torch.device("mps")
torch.set_default_device(device)


samples = pandas.read_csv("./train.csv")
samples_labels = torch.tensor(samples["label"].to_numpy()).long()
samples_feat = torch.tensor(samples.iloc[:, 1:].to_numpy()).float().view(-1, 28, 28)

subs = pandas.read_csv("./test.csv")
sub_feat = torch.tensor(subs[:].to_numpy()).float().view(-1, 28, 28)

train_size = int(samples_feat.shape[0] * 0.9)
train_feat = samples_feat[:train_size,:]
train_labels = samples_labels[:train_size]
test_feat = samples_feat[train_size:,:]
test_labels = samples_labels[train_size:]

print(samples.shape, train_feat.shape, train_labels.shape, test_feat.shape, test_labels.shape)


(42000, 785) torch.Size([37800, 28, 28]) torch.Size([37800]) torch.Size([4200, 28, 28]) torch.Size([4200])


In [11]:
    
class MnistConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 16, 5, padding="same")
        self.conv2 = torch.nn.Conv2d(16, 16, 5, padding="same")
        n = 28 * 28 // 4 // 4 * 16
        self.dense1 = torch.nn.Linear(n, n)
        self.dense2 = torch.nn.Linear(n, n)
        self.logits = torch.nn.Linear(n, 10)

    def forward(self, x):
        x = x.unsqueeze(1)
        v = self.conv1(x)
        v = F.relu(v)
        v = F.max_pool2d(v, 2)
        v = self.conv2(v)
        v = F.relu(v)
        v = F.max_pool2d(v, 2)
        v = torch.flatten(v, 1)
        v = self.dense1(v)
        v = F.tanh(v)
        v = self.dense2(v)
        v = F.tanh(v)
        v = self.logits(v)
        return F.log_softmax(v)

In [12]:
learning_rate = 1e-4
classifier = MnistConv()
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)
batch_size = 1500

for epoch in range(75):
    classifier.train()
    for i in range((train_feat.shape[0]//batch_size)):
        optimizer.zero_grad()
        batch_samples = train_feat[i * batch_size:(i + 1) * batch_size, :]
        batch_labels = train_labels[i * batch_size:(i + 1) * batch_size]
        # print(batch_samples)
        outputs = classifier(batch_samples)
        loss = F.cross_entropy(outputs, batch_labels)
        loss.backward()
        optimizer.step()

    classifier.eval()
    predicted_labels = torch.argmax(classifier(test_feat), 1)
    corrects = torch.eq(predicted_labels, test_labels).float().sum()
    print(f'accuracy (epoch {epoch}): {corrects/predicted_labels.shape[0]:5f}')

classifier.eval()
sub_pred = torch.argmax(classifier(sub_feat), 1)
sub_ids = torch.arange(1, sub_feat.shape[0] + 1)

results = pandas.DataFrame()
results["ImageId"] = sub_ids.cpu().numpy()
results["Label"] = sub_pred.long().cpu().numpy()
results.to_csv("./results.csv", index=False)

/Users/jturunen/repos/mnist-vae/.conda/lib/python3.10/site-packages/torch/utils/_device.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return func(*args, **kwargs)


accuracy (epoch 0): 0.885238
accuracy (epoch 1): 0.930000
accuracy (epoch 2): 0.946667
accuracy (epoch 3): 0.958333
accuracy (epoch 4): 0.965000
accuracy (epoch 5): 0.966190
accuracy (epoch 6): 0.972619
accuracy (epoch 7): 0.974762
accuracy (epoch 8): 0.975476
accuracy (epoch 9): 0.976905
accuracy (epoch 10): 0.977381
accuracy (epoch 11): 0.979762
accuracy (epoch 12): 0.980238
accuracy (epoch 13): 0.982619
accuracy (epoch 14): 0.982857
accuracy (epoch 15): 0.983095
accuracy (epoch 16): 0.982381
accuracy (epoch 17): 0.981667
accuracy (epoch 18): 0.981429
accuracy (epoch 19): 0.982857
accuracy (epoch 20): 0.983810
accuracy (epoch 21): 0.984048
accuracy (epoch 22): 0.984762
accuracy (epoch 23): 0.983810
accuracy (epoch 24): 0.984286
accuracy (epoch 25): 0.984048
accuracy (epoch 26): 0.984286
accuracy (epoch 27): 0.983810
accuracy (epoch 28): 0.983810
accuracy (epoch 29): 0.984762
accuracy (epoch 30): 0.984762
accuracy (epoch 31): 0.984524
accuracy (epoch 32): 0.984286
accuracy (epoch 33):